<!-- Notebook Header Start -->

<h1 align="center">Spike Sorting</h1>

<p align="center">
  <strong>Author:</strong> Karl Bates<br>
  <strong>Date:</strong> 2024-10-30<br>
  <strong>Affiliation:</strong> Carnegie Mellon University, Cohen-Karni Group  || Neuromechatronics Lab
</p>

---

## 📜 Project Overview

(to populate) 

- **Objective:** (to populate)
- **Scope:** (to populate)
- **Libraries:** `Python`, `Pandas`, `SciPy`

---

## 📊 Notebook Outline

1. **Data Import & Preprocessing**


---

## 📚 References & Additional Resources

- [Github: Kilosort4](https://github.com/MouseLand/Kilosort/tree/main)
- [SpikeInterface](https://github.com/SpikeInterface)

---

<!-- Notebook Header End -->


# Libraries

In [ ]:
# Imports
from pathlib import Path
from tqdm import tqdm
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd

from probeinterface import write_prb
# Download channel maps for default probes
from kilosort.utils import download_probes
from probeinterface.plotting import plot_probe, plot_probe_group
from probeinterface import Probe, get_probe, generate_linear_probe

from spikeinterface.extractors import read_intan

from kilosort import run_kilosort
from kilosort import io

# File Definitions

In [ ]:
# Parameters
RAT_NAME = 'DW322'
STIMULATION = 'DRGS_2_240918_130024'
DATA_DIRECTORY = Path(fr'D:\Data\CMU.80 Data\88 Analyzed Data\88.001 Kilosort on DW322')  # NOTE Specify the path where the data will be copied to, and where Kilosort4 results will be saved.
# Create path if it doesn't exist
DATA_DIRECTORY.mkdir(parents=True, exist_ok=True)

# Data Import

### defining filepath for data

In [ ]:
# NOTE: You will need to select the appropriate data stream. If you run without
#       specifying `stream_id`, you will get an error message explaining what
#       each stream corresponds to.
filepath = Path(f"D:\Data\CMU.80 Data\82 External Data\82.002 Sample Rat Data from RM1 Project\DW322\{RAT_NAME}\{STIMULATION}\{STIMULATION}.rhd")
recording = read_intan(filepath, stream_id='0')

no probe information is given with this data, so we will need to define it or assign one from the probe interface library.

I will first show how to create the binary file using `Spike Interface` and numpy's `memmap()`

## Import data and probe using `SpikeInterface`

In [ ]:
# NOTE: Data will be saved as np.int16 by default since that is the standard
#       for ephys data. If you need a different data type for whatever reason
#       such as `np.uint16`, be sure to update this.
dtype = np.int16
filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(
    recording, DATA_DIRECTORY, data_name=f'{RAT_NAME}_{STIMULATION}_data.bin', dtype=dtype,
    chunksize=60000, export_probe=True, probe_name='probe.prb'
    )

this code will complete, but because there is no probe associated with the data, I will need to define one later on

## Import data and probe using `Numpy memmap`

### defining ouput file and loading data into binary in chunks:

In [ ]:
# Define output binary file
output_file = DATA_DIRECTORY / f'{RAT_NAME}_{STIMULATION}_data_memmap.bin'

# Get data shape
num_channels = recording.get_num_channels()
num_samples = recording.get_num_frames()

# Create a memory-mapped file without using `with`
data_dtype = np.int16  # This is typically the standard for electrophysiology data
memmap_file = np.memmap(output_file, dtype=data_dtype, mode='w+', shape=(num_samples, num_channels))

# Define the chunk size for processing (e.g., 60,000 samples at a time)
chunk_size = 60000

# Loop through the data and write to the binary file in chunks
for start_idx in range(0, num_samples, chunk_size):
    end_idx = min(start_idx + chunk_size, num_samples)
    
    # Extract data chunk from the RecordingExtractor
    data_chunk = recording.get_traces(start_frame=start_idx, end_frame=end_idx)
    
    # Write the chunk to the memory-mapped file
    memmap_file[start_idx:end_idx, :] = data_chunk.astype(data_dtype)

# Explicitly flush changes to disk
memmap_file.flush()

print(f"Data successfully saved to {output_file}")


### exporting probe data

this will NOT run correctly, since there is no probe data

In [ ]:

# Get the probe from the recording (if available)
probe = recording.get_probe()

# Define the path to save the probe configuration
probe_path = DATA_DIRECTORY / 'probe.prb'

# Save the probe configuration
write_prb(probe_path, probe)


# Defining or assigning a probe

## downloading standard probes

you can use the `ProbeInterface` repository to define a standard probe.

You can then pull a probe from the `ProbeInterface` library

In [ ]:
manufacturer = 'neuronexus'
probe_name = 'A1x32-Poly3-10mm-50-177'

probe = get_probe(manufacturer, probe_name)
print(probe)

but, this is not the probe we are using. Instead, we are using a `A1x32-Edge-5mm-20-177-A32`

This is not in the library, so I will have to

## defining a probe

the following parameters came from this website:

[Neuronexus Penetrating Probe Catalog](https://19885756.fs1.hubspotusercontent-na1.net/hubfs/19885756/Penetrating_probe_catalog_V1.8.pdf#page=1.06&gsr=0)

Using the spreadsheet, `Adapter_pinout.xlsx`, the contact ID's can be traced to the "device channel", and we can assign them on the probe. 

In this case, our channel indices correspond to the aux inputs to the intan headstage.

In [ ]:
channel_mapping = pd.read_excel("Adapter_pinout.xlsx",sheet_name="summary",header=2).iloc[:,[8,10,12,14]]
channel_mapping

the channel indices shall be the `Intan.1` column

In [ ]:
linear_probe = generate_linear_probe(num_elec=32, ypitch=20)

# Define the number of channels and their positions along the probe shank
num_channels = 32
positions = [
    (0, i * 20) for i in range(num_channels)
]  # Channel spacing in micrometers

# Create a Probe object
linear_probe = Probe(ndim=2, si_units='um')
linear_probe.set_contacts(positions=positions, shapes='rect', shape_params={'width': 18,"height":10})

# Assign electrical connections based on the connector diagram
linear_probe.set_contact_ids(np.arange(1, 33)) # I set the first input to 1, instead of Python's traditional 0, to match the connector diagram


linear_probe.set_device_channel_indices(channel_mapping['Intan Input.1'][::-1]) # the [::-1] is to reverse the order

# first plot: contact ids on the probe itself
plot_probe(linear_probe, with_device_index=False,with_contact_id=True)

# second plot: device ids on the probe; when the intan reads, say, "dev14", it means that it's the 1st input on the electrode probe (in graph 1)
plot_probe(linear_probe, with_device_index=True,with_contact_id=False)


In [ ]:
linear_probe.to_dataframe()

### saving the probe alongside the raw data file:

In [ ]:
# Save the new probe to a .prb file for Kilosort
probe_path = DATA_DIRECTORY / '`A1x32-Edge-5mm-20-177-A32`.prb'
write_prb(probe_path, linear_probe)

print(f"Probe saved to {probe_path}")

In [ ]:
print(probe.device_channel_indices)

In [ ]:
# Connection Mapping: A32 Male -> O32 Connector
# Define the mapping from A32 Female to O32 as described
# Format: O32 Channel -> A32 Male Channel
o32_to_a32_mapping = [
    ("R", None), (23, 10), (25, 8), (27, 6), (29, 4), (31, 2),
    (19, 1), (17, 3), (21, 5), (11, 7), (15, 9), (13, 11), (1, 13), (3, 15), (5, 17), (7, 19), (9, 21),
    ("G", 23), (24, 12), (26, 14), (28, 16), (30, 18), (32, 20),
    (20, 22), (18, 24), (22, 26), (12, 28), (16, 30), (14, 32), (2, None), (4, None), (6, None), (8, None), (10, None),
    ("R", None)
]

# Set the device channel indices using the mapping
device_channel_indices = [a32_ch if a32_ch is not None else -1 for _, a32_ch in o32_to_a32_mapping]
probe.set_device_channel_indices(device_channel_indices)

# # Annotate ground and reference channels with full-length lists
# ground_channels = [ch == 23 for ch in range(32)]  # True for channel 23, False otherwise
# reference_channels = [False] * 32  # No reference channels specified in this example

# probe.annotate_contacts(ground_channels=ground_channels, reference_channels=reference_channels)

# Visualize and Save
import matplotlib.pyplot as plt

plot_probe(probe, with_contact_id=True)

# Save the probe definition
# probe.save_to_file("A1x32-Edge-5mm-20-177-A32-O32.json")

In [ ]:
probe.device_channel_indices

In [ ]:
# Save the new probe to a .prb file for Kilosort
probe_path = DATA_DIRECTORY / 'custom_probe.prb'
write_prb(probe_path, probe_group)

# Running Kilosort4

At this point, it's a good idea to open the Kilosort gui and check that the
data and probe appear to have been loaded correctly and no settings need to be
tweaked. You will need to input the path to the binary datafile, the folder where
results should be saved, and select a probe file.

```conda activate kilosort```

```python -m kilosort```

From there, you can either launch Kilosort using the GUI or run the
next notebook cell to run it through the API.

In [ ]:
# NOTE: 'n_chan_bin' is a required setting, and should reflect the total number
#       of channels in the binary file. For information on other available
#       settings, see `kilosort.run_kilosort.default_settings`.
settings = {'data_dir': DATA_DIRECTORY, 'n_chan_bin': 34}

ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = \
    run_kilosort(
        settings=settings, probe_name='neuropixPhase3B1_kilosortChanMap.mat',
        # save_preprocessed_copy=True
        )   